In [1]:
#r "nuget:Microsoft.DotNet.Interactive.AI, 1.0.0-beta.23558.10"

Installed Packages Microsoft.DotNet.Interactive.AI, 1.0.0-beta.23468.5

Loading extension script from `C:\Users\dicolomb\.nuget\packages\duckdb.interactiveextension\1.0.108\interactive-extensions\dotnet\extension.dib`

Query DuckDB databases. 
 This extension adds support for connecting to DuckDB databases using the #!connect duckdb magic command. ADO.NET support via DuckDB.NET project

Loading extension script from `C:\Users\dicolomb\.nuget\packages\microsoft.dotnet.interactive.ai\1.0.0-beta.23468.5\interactive-extensions\dotnet\extension.dib`

Added magic commands
 - `#!connect azure-openai`
 - `#!connect openai`
 - `#!connect huggingface`

Loading extensions from `C:\Users\dicolomb\.nuget\packages\skiasharp\2.88.5\interactive-extensions\dotnet\SkiaSharp.DotNet.Interactive.dll`

In [2]:
#!value --name key
YOUR AZURE OPEN AI KEY

In [3]:
#!value --name endpoint
https://your-enpoint.openai.azure.com/

In [4]:
#!connect azure-openai --model-type TextEmbeddingGenerator --kernel-name knowledge --api-key @value:key --endpoint @value:endpoint --deployment text-embedding-ada-002

Kernel added: #!knowledge(duckDbVectorRAW)

Kernel added: #!knowledge(duckDbVector)

Kernel added: #!knowledge

In [5]:
#!connect azure-openai --model-type TextCompletion --kernel-name textCompletion --api-key @value:key --endpoint @value:endpoint --deployment text-davinci-003 --use-knowledge knowledge

Kernel added: #!textCompletion(skill)

Kernel added: #!textCompletion

# `#!graph` command

## define prompt template

In [6]:
#!function generate_sequence_diagram --skill explain_code --description "generate mermaid sequence diagram from code"
[BEGIN TEXT]
{{$input}}
[END TEXT]
write mermaid sequence diagram markdown for the code above. remove all code fence markers

## test it 

In [7]:
#!use-skills function.explain_code.generate_sequence_diagram

public static int Fibonacci(int n)
{
    if (n <= 1)
    {
        return n;
    }
    else
    {
        return Fibonacci(n - 1) + Fibonacci(n - 2);
    }
}



sequenceDiagram
    participant A as Fibonacci(n)
    participant B as Fibonacci(n-1)
    participant C as Fibonacci(n-2)
    A->>B: n-1
    A->>C: n-2
    B->>A: Fibonacci(n-1)
    C->>A: Fibonacci(n-2)
    A->>A: Fibonacci(n)

## integrate in `smart app`

In [8]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Events;
using Microsoft.DotNet.Interactive.Commands;
using Microsoft.DotNet.Interactive.AI;
using Microsoft.DotNet.Interactive.Mermaid;
using System.CommandLine;

var explainMagicCommand = new Command("#!graph");
var textCompletionKernel = Kernel.Root.FindKernels(k => k is TextCompletionKernel).Single(t => t.Name == "textCompletion");
var mermaidKernel = Kernel.Root.FindKernels(k => k is MermaidKernel).Single();
Kernel.Root.AddDirective(explainMagicCommand);

In [9]:
explainMagicCommand.SetHandler(async _ =>
{
    var context = KernelInvocationContext.Current;
    if (context.Command is SubmitCode submitCode)
    {       

        textCompletionKernel.SetSuppressDisplay(true);
        List<KernelEvent> events = new();
        var prompt = $"""
            #!use-skills function.explain_code.generate_sequence_diagram

            {submitCode.Code.Replace("#!graph", "")}       
                 
            """;
        var sub = Kernel.Root.KernelEvents.Subscribe(e => events.Add(e));
        var results = await textCompletionKernel.SendAsync(new SubmitCode(prompt, textCompletionKernel.Name));
        
        sub.Dispose();
        textCompletionKernel.SetSuppressDisplay(false);
        var returnValueProduced = events
            .OfType<ReturnValueProduced>()
            .FirstOrDefault(e => e.Command.TargetKernelName == textCompletionKernel.Name);

        var markdown = returnValueProduced?
            .FormattedValues
            .OfType<FormattedValue>()
            .Single(fm => fm.MimeType == "text/plain")
            .Value
            .ToString() ?? string.Empty;

        // send to mermaid kernel for imemdiate display
        await mermaidKernel.SendAsync(new SubmitCode(markdown.Replace("```mermaid", string.Empty).Replace("```", string.Empty)));

        context.Complete(submitCode);
    }
});

## try the app experience

In [10]:
#!graph
public static int Fibonacci(int n)
{
    if (n <= 1)
    {
        return n;
    }
    else
    {
        return Fibonacci(n - 1) + Fibonacci(n - 2);
    }
}


## export the code